In [1]:
import pickle
filename = 'GridSearchCv/1-3 HOG SVM scaled rbf C=10.sav'
SVM = pickle.load(open(filename, 'rb'))

/Users/abhinavsaurabh/opt/anaconda3/lib/python3.8/site-packages/sklearn/base.py:329: UserWarning: Trying to unpickle estimator SVC from version 0.22.2.post1 when using version 0.23.1. This might lead to breaking code or invalid results. Use at your own risk.
  warnings.warn(


In [2]:
CIFAR = 'cifar-10-batches-py/'
import numpy as np

In [3]:
def unpickle(file):
    import pickle
    with open(file,'rb') as fo:
        cifar_dict = pickle.load(fo,encoding='bytes')
    return cifar_dict

In [4]:
dirs = ['batches.meta','data_batch_1','data_batch_2','data_batch_3','data_batch_4','data_batch_5','test_batch']

all_data = [0,1,2,3,4,5,6]

for i, direc in zip(all_data,dirs):
    all_data[i] = unpickle(CIFAR+direc)

In [5]:
batch_meta = all_data[0]
data_batch1 = all_data[1]
data_batch2 = all_data[2]
data_batch3 = all_data[3]
data_batch4 = all_data[4]
data_batch5 = all_data[5]
test_batch = all_data[6]

In [6]:
all_train_batches = [data_batch1,data_batch2,data_batch3,data_batch4,data_batch5]
test_batch1 = [test_batch]
#all_train_batches = [data_batch1]
#all_train_batches1 = all_train_batches[:1000]
training_image = np.vstack([d[b"data"] for d in all_train_batches])
train_len = len(training_image)
training_image = training_image.reshape(train_len,3*32*32)
training_label = np.hstack([d[b"labels"] for d in all_train_batches])
training_label = training_label.reshape(50000)

#temp = training_image[:100]
#temp_2  = training_label[:100]

In [7]:
test_image = np.vstack([d[b"data"] for d in test_batch1])
test_len = len(test_image)
test_image = test_image.reshape(test_len,3*32*32)
test_label = np.hstack([d[b"labels"] for d in test_batch1])

In [10]:
import numpy as np
import matplotlib.colors as colors
from skimage.feature import hog
from skimage import color

X_train_hog = []

for img in training_image:

    img = np.reshape(img.T,(32,32,3))
    # color histogram
    array=np.asarray(img)
    arr=(array.astype(float))/255.0
    img_hsv = colors.rgb_to_hsv(arr[...,:3])
    img_color_hist = np.histogram(img_hsv[...,0],bins=10)
    hist_values = img_color_hist[0]

    #hog
    gray_image = color.rgb2gray(img)
    hog_feature = hog(gray_image,orientations=8, pixels_per_cell=(8, 8), cells_per_block=(1, 1))

    #joined
    X_train_hog.append(np.concatenate((hist_values,hog_feature),axis=None))

In [12]:
X_test_hog = []

for img in test_image:

    img = np.reshape(img.T,(32,32,3))
    # color histogram
    array=np.asarray(img)
    arr=(array.astype(float))/255.0
    img_hsv = colors.rgb_to_hsv(arr[...,:3])
    img_color_hist = np.histogram(img_hsv[...,0],bins=10)
    hist_values = img_color_hist[0]

    #hog
    gray_image = color.rgb2gray(img)
    hog_feature = hog(gray_image,orientations=8, pixels_per_cell=(8, 8), cells_per_block=(1, 1))

    #joined
    X_test_hog.append(np.concatenate((hist_values,hog_feature),axis=None))

In [13]:
import numpy
X_train =numpy.asarray(X_train_hog)
X_test = numpy.asarray(X_test_hog)

In [14]:
filename = 'XTrain_HOG 50000.sav'
pickle.dump(X_train, open(filename, 'wb'))

In [15]:
filename = 'XTest_HOG 10000.sav'
pickle.dump(X_test, open(filename, 'wb'))

In [8]:
filename = 'XTrain_HOG 50000.sav'
X_train = pickle.load(open(filename, 'rb'))

In [9]:
filename = 'XTest_HOG 10000.sav'
X_test = pickle.load(open(filename, 'rb'))

In [17]:
from sklearn import preprocessing
X_train = preprocessing.scale(X_train)
X_test = preprocessing.scale(X_test)

In [18]:
X_train.shape

(50000, 138)

In [19]:
SVM.support_vectors_.shape

(48970, 138)

In [20]:
x_m2 = X_train[SVM.support_]
y_m2 = training_label[SVM.support_]

In [21]:
from sklearn.svm import SVC 
classifier = SVC(kernel ='rbf', random_state = 0,C=10) 
 # training set in x, y axis 
classifier.fit(x_m2, y_m2)

SVC(C=10, random_state=0)

In [27]:
filename = '1-4 HOG classifier.sav'
pickle.dump(classifier, open(filename, 'wb'))

In [22]:
trainpredict = classifier.predict(x_m2) 

In [23]:
testpredict = classifier.predict(X_test)

In [24]:
from sklearn.metrics import classification_report, confusion_matrix
print(classification_report(y_m2, trainpredict))

              precision    recall  f1-score   support

           0       1.00      0.99      0.99      4872
           1       0.96      0.98      0.97      4847
           2       0.99      0.95      0.97      4968
           3       0.96      0.95      0.96      4991
           4       0.96      0.94      0.95      4914
           5       0.94      0.96      0.95      4923
           6       0.89      0.97      0.93      4648
           7       0.97      0.96      0.97      4905
           8       1.00      0.98      0.99      4938
           9       0.97      0.96      0.97      4964

    accuracy                           0.96     48970
   macro avg       0.96      0.96      0.96     48970
weighted avg       0.96      0.96      0.96     48970



In [25]:
print(classification_report(test_label, testpredict))

              precision    recall  f1-score   support

           0       0.43      0.49      0.46      1000
           1       0.39      0.44      0.41      1000
           2       0.31      0.27      0.29      1000
           3       0.23      0.21      0.22      1000
           4       0.34      0.32      0.33      1000
           5       0.32      0.33      0.32      1000
           6       0.42      0.44      0.43      1000
           7       0.32      0.30      0.31      1000
           8       0.44      0.45      0.45      1000
           9       0.30      0.28      0.29      1000

    accuracy                           0.35     10000
   macro avg       0.35      0.35      0.35     10000
weighted avg       0.35      0.35      0.35     10000

